# 비속어 필터링

https://github.com/sgunderscore/hatescore-korean-hate-speech/tree/main

In [ ]:
from transformers import TextClassificationPipeline, BertForSequenceClassification, AutoTokenizer
model_name = 'sgunderscore/hatescore-korean-hate-speech'
model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
pipe = TextClassificationPipeline(
        model = model,
        tokenizer = tokenizer,
        device = -1, # gpu: 0
        return_all_scores = True,
        function_to_apply = 'sigmoid')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/722k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
pipe("꼭 키 작은 급식충이 이런 글 씀")[0]

[{'label': 'None', 'score': 0.07644891738891602},
 {'label': '기타 혐오', 'score': 0.08226024359464645},
 {'label': '남성', 'score': 0.03568856045603752},
 {'label': '단순 악플', 'score': 0.04831737279891968},
 {'label': '성소수자', 'score': 0.02385660447180271},
 {'label': '여성/가족', 'score': 0.06244980916380882},
 {'label': '연령', 'score': 0.9193087816238403},
 {'label': '인종/국적', 'score': 0.051288578659296036},
 {'label': '종교', 'score': 0.019473949447274208},
 {'label': '지역', 'score': 0.02318374067544937}]

In [ ]:
text = "야 이 병신같은 새끼야. 대가리를 왜 달고 다님?"
# text = '안녕하세요'
label_and_score = pipe(text)[0]

label_and_score = sorted(label_and_score, key=lambda x:-x['score'])

if label_and_score[0]['label'] == 'None':
  print('정상 댓글')

else:
  print('비속어 감지')
  print(f'{label_and_score[0]["label"]} 비하 글이므로 필터링 필요.')

비속어 감지
단순 악플 비하 글이므로 필터링 필요.


In [ ]:
!pip install transformers

In [ ]:
import re
import torch
from transformers import BertTokenizer, BertForTokenClassification

# KoBERT 토크나이저 및 모델 로드
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = BertForTokenClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 주민등록번호와 전화번호 패턴 정의
jumin_pattern = r'(\d{6}-\d{7})'
phone_pattern = r'(\d{3}-\d{4}-\d{4})'

# 주민등록번호와 전화번호 필터링 함수
def filter_sensitive_info(text):
    # 주민등록번호와 전화번호 식별하여 레이블 설정
    labels = [0] * len(text)  # 개인정보가 아닌 경우는 0, 개인정보인 경우는 1
    for match in re.finditer(jumin_pattern, text):
        start, end = match.span()
        for i in range(start, end):
            labels[i] = 1
    for match in re.finditer(phone_pattern, text):
        start, end = match.span()
        for i in range(start, end):
            labels[i] = 1

    # 텍스트를 토큰화하여 BERT 입력 형식으로 변환
    tokens = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
    segments_ids = [0] * len(tokens)

    # BERT 입력 형식으로 변환
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    # 개인정보 필터링을 위한 BERT 모델 실행
    with torch.no_grad():
        outputs = model(tokens_tensor, token_type_ids=segments_tensors)

    # 마스킹된 텍스트 생성
    masked_text = ""
    for token, label in zip(tokens, outputs[0].argmax(2)[0]):
        if label.item() == 1:  # 개인정보로 분류된 토큰
            masked_text += "*"
        else:
            masked_text += token
        masked_text += " "

    return masked_text.strip()

# 예시 텍스트
text = "주민등록번호는 950101-1234567이고, 전화번호는 010-1234-5678입니다."

# 주민등록번호와 전화번호 필터링 함수 호출
filtered_text = filter_sensitive_info(text)
print("Original text:", text)
print("Filtered text:", filtered_text)

Original text: 주민등록번호는 950101-1234567이고, 전화번호는 010-1234-5678입니다.
Filtered text: 주 ##민 ##등 ##록 ##번 ##호는 950 ##10 ##1 - 1234 ##5 * ##7 ##이고 , 전 ##화 ##번 ##호는 010 - 1234 - * * ##입 ##니다 .


In [ ]:
# 주민등록번호와 전화번호 패턴 정의
jumin_pattern = r'(\d{6}-\d{7})'
phone_pattern = r'(\d{3}-\d{4}-\d{4})'

# 예시 텍스트
text = "주민등록번호는 950101-1234567이고, 전화번호는 010-1234-5678입니다."

labels = [0] * len(text)  # 개인정보가 아닌 경우는 0, 개인정보인 경우는 1
for match in re.finditer(jumin_pattern, text):
    start, end = match.span()
    for i in range(start, end):
        labels[i] = 1
for match in re.finditer(phone_pattern, text):
    start, end = match.span()
    for i in range(start, end):
        labels[i] = 1

print(labels)
tokens = tokenizer.tokenize(text)

tmp = []
tmp_str=str()
for i in tokens:
  if i[:2]=='##':
    tmp_str+=i[2:]
  else:
    tmp.append(tmp_str)
    tmp_str = i
print(tmp)

indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
segments_ids = [0] * len(tokens)

print(tokens)
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

print(tokens_tensor, segments_tensors)

with torch.no_grad():
  outputs = model(tokens_tensor, token_type_ids=segments_tensors)

print(outputs)
print(len(outputs), len(indexed_tokens))

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
['', '주민등록번호는', '950101', '-', '1234567이고', ',', '전화번호는', '010', '-', '1234', '-', '5678입니다']
['주', '##민', '##등', '##록', '##번', '##호는', '950', '##10', '##1', '-', '1234', '##5', '##6', '##7', '##이고', ',', '전', '##화', '##번', '##호는', '010', '-', '1234', '-', '567', '##8', '##입', '##니다', '.']
tensor([[  9689,  36553, 101322,  31398,  35465, 100543,  29997,  20305,  10759,
            118,  82749,  11166,  11211,  11305,  54355,    117,   9665,  18227,
          35465, 100543,  49470,    118,  82749,    118,  52603,  11396,  58303,
          48345,    119]]) tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0]])
TokenClassifierOutput(loss=None, logits=tensor([[[ 0.3802, -0.1070],
         [ 0.1866, -0.2390],
         [ 0.1157, -0.1736],
         [ 0.1351, -0.1179],
         [ 0.0744, -0.0576],

In [ ]:
import re

def mask_personal_info(text):
    # 이메일 주소 마스킹
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    masked_text = re.sub(email_pattern, '***@***.***', text)

    # 전화번호 마스킹
    phone_pattern = r'(\d{3})-(\d{3,4})-(\d{4})'
    masked_text = re.sub(phone_pattern, r'\1-****-\3', masked_text)

    # 주민등록번호 마스킹
    resident_id_pattern = r'(\d{6})-?([1-4])(\d{6})'  # 뒷자리 6자리를 추출하기 위한 패턴
    masked_text = re.sub(resident_id_pattern, r'\1-\2******', masked_text)

    return masked_text

# 테스트용 문자열
sample_text = """
개인 정보 보호를 위해 전화번호 010-1234-5678와 이메일 test@example.com,
그리고 주민등록번호 920101-1234567을 마스킹합니다.
"""

# 마스킹된 결과 출력
print(mask_personal_info(sample_text))



개인 정보 보호를 위해 전화번호 010-****-5678와 이메일 ***@***.***,
그리고 주민등록번호 920101-1******을 마스킹합니다.



In [ ]:
import re

def mask_personal_info(text):
    # 이메일 주소 마스킹
    email_pattern = r'\b([A-Za-z0-9._%+-]{2})([A-Za-z0-9._%+-]+)@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    masked_text = re.sub(email_pattern, r'\1**@\2', text)

    # 전화번호 마스킹
    phone_pattern = r'(\d{3})-(\d{4})-(\d{4})'
    masked_text = re.sub(phone_pattern, r'\1-\2-****', masked_text)

    # 주민등록번호 마스킹
    resident_id_pattern = r'(\d{6})-?([1-4])(\d{6})'
    masked_text = re.sub(resident_id_pattern, r'\1-\2******', masked_text)

    # 주민등록번호, 전화번호, 이메일 추출
    extracted_phone_numbers = re.findall(phone_pattern, text)
    extracted_resident_ids = re.findall(resident_id_pattern, text)
    extracted_emails = re.findall(email_pattern, text)

    return masked_text, extracted_resident_ids, extracted_phone_numbers, extracted_emails

# 테스트용 문자열
sample_text = """
개인 정보 보호를 위해 전화번호 010-1234-5678와 이메일 test@example.com,
그리고 주민등록번호 920101-1234567을 마스킹합니다.
"""

# 마스킹된 결과와 추출된 주민등록번호, 핸드폰번호, 이메일 출력
masked_result, resident_ids, phone_numbers, emails = mask_personal_info(sample_text)
print("마스킹된 결과:", masked_result)
print('추출된 주민등록번호', resident_ids)
print("추출된 핸드폰번호:", phone_numbers)
print("추출된 이메일:", emails)


마스킹된 결과: 
개인 정보 보호를 위해 전화번호 010-1234-****와 이메일 te**@st,
그리고 주민등록번호 920101-1******을 마스킹합니다.

추출된 주민등록번호 [('920101', '1', '234567')]
추출된 핸드폰번호: [('010', '1234', '5678')]
추출된 이메일: [('te', 'st')]


In [ ]:
import hashlib
import secrets

# 사용자의 비밀번호를 해시화하는 함수
def hash_password(password, salt=None):
    if salt is None:
        salt = secrets.token_hex(16)  # 16바이트(32자리16진수)의 무작위 salt 생성
    password = password.encode('utf-8')
    salt = salt.encode('utf-8')
    hashed_password = hashlib.sha256(password + salt).hexdigest()
    return hashed_password, salt.decode('utf-8')  # 소금을 다시 UTF-8 문자열로 디코딩해서 반환

# 비밀번호 검증 함수
def verify_password(password, hashed_password, salt):
    password, _ = hash_password(password, salt)
    return password == hashed_password

resident__ids = f'{resident_ids[0][0]}-{resident_ids[0][1]}{resident_ids[0][2]}'
print(resident__ids)
hashed_password_resident, salt_resident = hash_password(resident__ids)
print('주민등록번호')
print(hashed_password_resident, salt_resident)

if verify_password('920101-1234567', hashed_password_resident, salt_resident):
    print("일치")
else:
    print("불일치")

print('-'*50)
phone__numbers = '-'.join(phone_numbers[0])
print(phone__numbers)
hashed_password_phone, salt_phone = hash_password(phone__numbers)
print('휴대폰번호')
print(hashed_password_phone, salt_phone)

if verify_password('010-1234-5678', hashed_password_phone, salt_phone):
    print("일치")
else:
    print("불일치")

print('-'*50)
_emails = emails[0]
print(_emails)
hashed_password_email, salt_email = hash_password(_emails)
print('이메일')
print(hashed_password_email, salt_email)

if verify_password('test@example.com', hashed_password_email, salt_email):
    print("일치")
else:
    print("불일치")

920101-1234567
주민등록번호
33b3ea03162e055a603bda059b9854d85e4398f6c177df30c0012d4e273e0abc a7d86c9c42443f3b79853fdc3827c184
일치
--------------------------------------------------
010-1234-5678
휴대폰번호
5baf10a39cd568e34f01f1bcf1d3e38de4c1aa611c14a86849cd5b34469dd8f8 46ce89397dae57dda674665b0f06cc1e
일치
--------------------------------------------------
test@example.com
이메일
f00cd550da704f8b19f86674adddfc02b930f69117eb029fecf8976c308e2617 4e7af8ef269f7efde7a5d59edf17f028
일치
